In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.reset_default_graph()

<h1>define the log directory</h1>

In [3]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "AND_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

<h1>Perceptron AND gate Example</h1>

In [4]:
#define number of training iterations
NUM_ITER = 2000
#define learning rate
lr = 0.01

In [5]:
#define the training input data
x = np.array([[0,0],[1,0],[1,1],[0,1]],np.float32)
#define the target outputs/labels for AND gate
y = np.array([0,0,1,0],np.float32)
#y = np.array([0,1,1,1],np.float32)       (uncomment this line if you are doing OR gate)
y = np.reshape(y,[4,1])

<h2>You define the loss scalar summary and the file writer</h2>

In [6]:
#define the placeholder to feed the training data later
X = tf.placeholder(tf.float32,shape=[4,2],name='X')
Y = tf.placeholder(tf.float32,shape=[4,1],name='Y')

#define the weights and initialized with random normal distribution
W = tf.Variable(tf.random_normal([2,1]),tf.float32,name='Weight')
#define the bias with zero initialization
b = tf.Variable(tf.zeros([1,1]),tf.float32,name='bias')

#define the predicted output
Y_hat = tf.add(tf.matmul(X,W),b,'Y_hat')

#define the loss function, we are using cross entropy function after sigmoid on Y_hat
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=Y_hat,labels=Y),name='loss')

#define the optimizer, here we use standard gradient descent method
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

#initialize tensorflow variables
init = tf.global_variables_initializer()

#!!!!!!!!!!!!!define loss scalar summary & File Writer!!!!!!!!
loss_summary = tf.summary.scalar('loss',loss)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [8]:
#start tensorflow session
with tf.Session() as sess:
    #run initialization
    sess.run(init)
    #training for number of iterations
    for i in range(NUM_ITER):
        if i%10 == 0:
            #fetch the loss every 10 iterations
            summary = sess.run(loss_summary,feed_dict={X:x,Y:y})
            #add the summary to file writer
            file_writer.add_summary(summary,i)
        #every iteration we run optimization on the training data X and labels Y
        sess.run(optimizer,feed_dict = {X:x,Y:y})
        
    #after training, we acquire the weight and bias
    #np.squeeze plays a role to get rid of the redundant dimension [i.e.make (2,1) be (2,)]
    W = np.squeeze(sess.run(W))
    b = np.squeeze(sess.run(b))

#remember to close file writer!
file_writer.close()

<p style="font-size:20px">Open tensorboard, you can see two things:</p>
<p style="font-size:20px">1. The graph

<img src="figures/tensorboard_2_graph.png">

<p style="font-size:20px">2. Next to the graphs, you can also visualize the scalars. (in this example, we define the summary of loss with training).

<img src="figures/tensorboard_2_scalar.png">